Setup to use either gpu or cpu.

In [2]:
import torch

device = "mps" if torch.backends.mps.is_available() else "cpu"
#device = "cuda" if torch.cuda.is_available() else "cpu" # <- uncomment for Windows
device

'mps'